In [2]:
import os
import re
import pandas as pd
import numpy as np
from os.path import join
%matplotlib inline
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Delist companies list
***

In [19]:
# Read halt information CSV
date = '20210611'
df_info = pd.read_csv('../../data/stocks_information/stocks_info-{}.csv'.format(date))

# Copy halt dataframe
df_delist = df_info.copy()

# Get delist stocks
# df_delist = df_delist[df_delist['listStatusCD'] == 'DE']
df_delist = df_delist[(df_delist['listStatusCD'] == 'DE') & (df_delist['ticker'].apply(lambda x: len(x) == 6))]

# Drop columns
df_delist = df_delist[['ticker', 'secShortName', 'delistDate']]

# Normalize ticker
df_delist['ticker'] = df_delist['ticker'].apply(lambda x: str(x).zfill(6))

# Delete duplicates (with different halt dates)
df_delist.drop_duplicates(inplace=True)

# Normalize ticker
df_delist = df_delist.rename(columns={
    'ticker': 'TICKER',
    'secShortName': 'STOCK_NAME',
    'delistDate': 'DELIST_DATE'
})
df_delist = df_delist.sort_values(by='TICKER')

# Save dataframe
df_delist.to_csv('../../data/stocks_information/delist-{}.csv'.format(date), index=False)
df_delist.to_excel('../../data/stocks_information/delist-{}.xlsx'.format(date), index=False)

df_delist.head()

,TICKER,STOCK_NAME,DELIST_DATE
2,000003,PT金田A,2002-06-14
12,000013,*ST石化A,2004-09-20
14,000015,PT中浩A,2001-10-22
17,000018,神城A退,2020-01-07
21,000022,深赤湾A,2018-12-26


In [4]:
df_delist.shape

(141, 3)

In [3]:
# # Read halt information CSV
# date = '20210417'
# df_halt = pd.read_csv('../../data/stocks_information/halt_{}.csv'.format(date))

# # Copy halt dataframe
# df_delist = df_halt.copy()

# # Get delist stocks
# df_delist = df_delist[df_delist['listStatusCD'] == 'DE']

# # Drop columns
# df_delist.drop(columns=['secID', 'haltBeginTime', 'haltEndTime', 'exchangeCD', 'listStatusCD', 'assetClass'], inplace=True)

# # Normalize ticker
# df_delist['ticker'] = df_delist['ticker'].apply(lambda x: str(x).zfill(6))

# # Delete duplicates (with different halt dates)
# df_delist.drop_duplicates(inplace=True)

# # Normalize ticker
# df_delist = df_delist.rename(columns={
#     'ticker': 'TICKER',
#     'secShortName': 'STOCK_NAME',
#     'delistDate': 'DELIST_DATE'
# })
# df_delist = df_delist.sort_values(by='TICKER')

# # Save dataframe
# df_delist.to_csv('../../data/stocks_information/delist_{}.csv'.format(date), index=False)
# df_delist.to_excel('../../data/stocks_information/delist_{}.xlsx'.format(date), index=False)

# df_delist.head()

/home/leander/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3166: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,TICKER,STOCK_NAME,DELIST_DATE
819,000018,神城A退,2020-01-07
1100,000024,招商地产,2015-12-30
1581,000033,新都退,2017-07-07
4094,000406,石油大明,2006-04-21
4631,000418,小天鹅A,2019-06-21


# ST companies list
***

In [5]:
# Read ST information CSV
date = '20210611'
df_st = pd.read_csv('../../data/stocks_information/st-{}.csv'.format(date), index_col=0)

# Normalize ticker
df_st['ticker'] = df_st['ticker'].apply(lambda x: str(x).zfill(6))

# Get the set of stocks
stocks = set(i for i in zip(df_st['ticker'], df_st['STflg'], df_st['tradeAbbrName']))

# Extract start date and end date of the ST duration
df_st_duration = pd.DataFrame(columns=('TICKER', 'STOCK_NAME', 'START_DATE', 'END_DATE', 'ST_FLAG'))
for t, s, n in stocks:
    df_i = df_st[(df_st['ticker'] == t) & (df_st['STflg'] == s)]
    dates_i = df_i['tradeDate'].sort_values().values
    start_date, end_date = dates_i[0], dates_i[-1]
    df_st_duration = df_st_duration.append(pd.DataFrame({
        'TICKER': [t], 
        'STOCK_NAME': [n], 
        'START_DATE': [start_date], 
        'END_DATE': [end_date], 
        'ST_FLAG': [s]
    }), ignore_index=True)
df_st_duration = df_st_duration.sort_values(by='TICKER')

# Save dataframe
df_st_duration.to_csv('../../data/stocks_information/st_duration-{}.csv'.format(date), index=False)
df_st_duration.to_excel('../../data/stocks_information/st_duration-{}.xlsx'.format(date), index=False)

df_st_duration.head()

In [ ]:
set(df_st['STflg'])

In [ ]:
print('S: ', len(set(df_st[df_st['STflg'] == 'S']['secID'])))
print('*: ', len(set(df_st[df_st['STflg'] == '*']['secID'])))
print('ST: ', len(set(df_st[df_st['STflg'] == 'ST']['secID'])))
print('*ST: ', len(set(df_st[df_st['STflg'] == '*ST']['secID'])))
print('SST: ', len(set(df_st[df_st['STflg'] == 'SST']['secID'])))
print('S*ST: ', len(set(df_st[df_st['STflg'] == 'S*ST']['secID'])))
print('Total: ', len(set(df_st['secID'])))


In [ ]:
orig_path = 'data/financial_statements_pit/original/csv'

for cn_name, en_name in self.cn_to_en.items():
    print(cn_name, en_name)
    df = pd.read_csv(join(orig_path, cn_name) + '.csv')

    # Nomalize ticker
    df['ticker'] = df['ticker'].apply(lambda x: str(x).zfill(6))

    # Delete Companies
    df.drop(df[df['ticker'].apply(
        lambda x: (x[0] in ['9', 'A', '2']) | (x in self.drop_list)
    )].index, inplace=True

    df = df.sort_values(
        by=['ticker', 'endDate', 'endDateRep', 'actPubtime', 'fiscalPeriod'], 
        ascending=[True, True, True, True, True]
    )

# Preprocessing
***

In [5]:
from preprocess import DataPreprocess

In [6]:
delete_list = set(df_delist['TICKER'])

DataPreprocess(
    '../../data/financial_statements/',
    '../../data/financial_statements/',
    delete_list
).process_l1()

DataPreprocess(
    '../../data/financial_statements/',
    '../../data/financial_statements/',
    delete_list
).process_l2()

income_bank
income_insurance
income_security
income_industry
balance_bank
balance_insurance
balance_security
balance_industry
cash_flow_bank
cash_flow_insurance
cash_flow_security
cash_flow_industry
----------------------------------------------------------------------
income_bank
----------------------------------------------------------------------
income_insurance
----------------------------------------------------------------------
income_security
----------------------------------------------------------------------
income_industry
----------------------------------------------------------------------
balance_bank
----------------------------------------------------------------------
balance_insurance
----------------------------------------------------------------------
balance_security
----------------------------------------------------------------------
balance_industry
----------------------------------------------------------------------
cash_flow_bank
---------------------

# Compare PIT

In [3]:
# import pandas as pd
# from os.path import join

# # Check if directories exit or not
# def check_dirs(path_list):
#     for dir_path in path_list:
#         if not os.path.isdir(dir_path):
#             os.makedirs(dir_path)

# df_cn_en = pd.read_csv('../../data/financial_statements/cn_to_en.csv')
# cn_to_en = {c: e for c, e in zip(df_cn_en['cn_name'], df_cn_en['en_name'])}

# save_path = '../../data/comparison'
# check_dirs([save_path])
# check_dirs([join(save_path, 'in-PIT-not-in-Non-PIT')])
# check_dirs([join(save_path, 'in-Non-PIT-not-in-PIT')])

# for cn_name, en_name in cn_to_en.items():
#     print('File:', en_name)

#     df_non_pit_orig = pd.read_csv('../../data/financial_statements/normalized_l2/csv/' + en_name + '.csv')
#     df_pit_orig = pd.read_csv('../../data/financial_statements_pit/normalized_l2/csv/' + en_name + '_pit.csv')

#     # Nomalize ticker
#     df_non_pit_orig['ticker'] = df_non_pit_orig['ticker'].apply(lambda x: str(x).zfill(6))
#     df_pit_orig['ticker'] = df_pit_orig['ticker'].apply(lambda x: str(x).zfill(6))
    
#     df_non_pit = df_non_pit_orig[['ticker', 'endDate']]
#     df_pit = df_pit_orig[['ticker', 'endDate']]

#     set_non_pit = set()
#     for _, row in df_non_pit.iterrows():
#         set_non_pit.add((
#             row['ticker'], 
#             row['endDate']
#         ))
#     set_pit = set()
#     for _, row in df_pit.iterrows():
#         set_pit.add((
#             row['ticker'], 
#             row['endDate']
#         ))

#     print('-' * 10)
#     print('Non-PIT shape:', df_non_pit.shape)
#     print('PIT shape:', df_pit.shape)
#     if set_non_pit - set_pit:
#         print('-' * 10)
#         print('In Non-PIT but not in PIT:')
#         df_in_non_pit = []
#         for i in set_non_pit - set_pit:
#             # print(i)
#             df_in_non_pit.append(
#                 df_non_pit_orig[(df_non_pit_orig['ticker'] == i[0]) & (df_non_pit_orig['endDate'] == i[1])])
#         df_in_non_pit = pd.concat(df_in_non_pit)
#         df_in_non_pit = df_in_non_pit.sort_values(by='ticker')
#         df_in_non_pit.to_excel(join(join(save_path, 'in-Non-PIT-not-in-PIT'), en_name + '.xlsx'), index=False)
#     if set_pit - set_non_pit:
#         print('-' * 10)
#         print('In PIT but not in Non-PIT:')
#         df_in_pit = []
#         for i in set_pit - set_non_pit:
#             # print(i)
#             df_in_pit.append(
#                 df_pit_orig[(df_pit_orig['ticker'] == i[0]) & (df_pit_orig['endDate'] == i[1])])
#         df_in_pit = pd.concat(df_in_pit)
#         df_in_pit = df_in_pit.sort_values(by='ticker')
#         df_in_pit.to_excel(join(join(save_path, 'in-PIT-not-in-Non-PIT'), en_name + '.xlsx'), index=False)
#     print('=' * 70)

File: income_bank
----------
Non-PIT shape: (1120, 2)
PIT shape: (1114, 2)
----------
In Non-PIT but not in PIT:
File: income_insurance
----------
Non-PIT shape: (216, 2)
PIT shape: (216, 2)
File: income_security
----------
Non-PIT shape: (1192, 2)
PIT shape: (1191, 2)
----------
In Non-PIT but not in PIT:
File: income_industry
----------
Non-PIT shape: (125766, 2)
PIT shape: (124398, 2)
----------
In Non-PIT but not in PIT:
----------
In PIT but not in Non-PIT:
File: balance_bank
----------
Non-PIT shape: (1033, 2)
PIT shape: (1033, 2)
File: balance_insurance
----------
Non-PIT shape: (188, 2)
PIT shape: (188, 2)
File: balance_security
----------
Non-PIT shape: (1121, 2)
PIT shape: (1122, 2)
----------
In PIT but not in Non-PIT:
File: balance_industry
----------
Non-PIT shape: (120154, 2)
PIT shape: (121376, 2)
----------
In PIT but not in Non-PIT:
File: cash_flow_bank
----------
Non-PIT shape: (1091, 2)
PIT shape: (1085, 2)
----------
In Non-PIT but not in PIT:
File: cash_flow_insura

# Find zeros
***

In [7]:
import pandas as pd
from os.path import join

# Check if directories exit or not
def check_dirs(path_list):
    for dir_path in path_list:
        if not os.path.isdir(dir_path):
            os.makedirs(dir_path)

data_path = '../../data/financial_statements/'

df_cn_en = pd.read_csv(join(data_path, 'cn_to_en.csv'))
cn_to_en = {c: e for c, e in zip(df_cn_en['cn_name'], df_cn_en['en_name'])}

save_path = join(data_path, 'normalized_l2/zeros')
check_dirs([save_path])

for en_name in cn_to_en.values():
    print('File:', en_name)
    df = pd.read_csv(join(join(data_path, 'normalized_l2/csv'), en_name) + '.csv')
    df_zero = []
    for _, row in df.iterrows():
        for k, v in row.items():
            if v == 0:
                df_zero.append(row)
    if df_zero:
        df_zero = pd.concat(df_zero, axis=1).T
        df_zero.to_excel(join(save_path, en_name) + '.xlsx', index=False)

File: income_bank
File: income_insurance
File: income_security
File: income_industry
File: balance_bank
File: balance_insurance
File: balance_security
File: balance_industry
File: cash_flow_bank
File: cash_flow_insurance
File: cash_flow_security
File: cash_flow_industry


In [7]:
# import pandas as pd
# from os.path import join

# # Check if directories exit or not
# def check_dirs(path_list):
#     for dir_path in path_list:
#         if not os.path.isdir(dir_path):
#             os.makedirs(dir_path)

# data_path = '../../data/financial_statements_pit/'

# df_cn_en = pd.read_csv(join(data_path, 'cn_to_en.csv'))
# cn_to_en = {c: e for c, e in zip(df_cn_en['cn_name'], df_cn_en['en_name'])}

# save_path = join(data_path, 'normalized_l2/zeros')
# check_dirs([save_path])

# for en_name in cn_to_en.values():
#     print('File:', en_name)
#     df = pd.read_csv(join(join(data_path, 'normalized_l2/csv'), en_name) + '.csv')
#     df_zero = []
#     for _, row in df.iterrows():
#         for k, v in row.items():
#             if v == 0:
#                 df_zero.append(row)
#     if df_zero:
#         df_zero = pd.concat(df_zero, axis=1).T
#         df_zero.to_excel(join(save_path, en_name) + '.xlsx', index=False)

# Check Q3
***

In [8]:
import pandas as pd
from os.path import join

# Check if directories exit or not
def check_dirs(path_list):
    for dir_path in path_list:
        if not os.path.isdir(dir_path):
            os.makedirs(dir_path)

def check_Q3_fiscal_period(data_path):
    df_cn_en = pd.read_csv(join(data_path, 'cn_to_en.csv'))
    cn_to_en = {c: e for c, e in zip(df_cn_en['cn_name'], df_cn_en['en_name'])}

    save_path = join(data_path, 'Q3-3-not-9_endDate_endDateRep')
    check_dirs([save_path])

    for en_name in cn_to_en.values():
        print('File:', en_name)
        df = pd.read_csv(join(join(data_path, 'original/csv'), en_name) + '.csv')
        df_Q3_3 = df[(df['reportType']=='Q3') | (df['reportType']=='CQ3')].groupby(
            ['ticker', 'endDate', 'endDateRep']).apply(
                lambda x: 9 not in set(x['fiscalPeriod']))
        df_Q3_3_not_9 = []
        for t, ed, edr in df_Q3_3[df_Q3_3.values].index.to_list():
            df_Q3_3_not_9.append(
                df[(df['ticker']==t) \
                & (df['endDate']==ed) \
                & (df['endDateRep']==edr) \
                & (df['mergedFlag']==1)])
        if df_Q3_3_not_9:
            df_Q3_3_not_9 = pd.concat(df_Q3_3_not_9)
            df_Q3_3_not_9.to_excel(join(save_path, en_name) + '.xlsx', index=False)

In [9]:
check_Q3_fiscal_period('../../data/financial_statements/')
# check_Q3_fiscal_period('../../data/financial_statements_pit/')

File: income_bank
File: income_insurance
File: income_security
File: income_industry
File: balance_bank
File: balance_insurance
File: balance_security
File: balance_industry
File: cash_flow_bank
File: cash_flow_insurance
File: cash_flow_security
File: cash_flow_industry


In [10]:
import pandas as pd
from os.path import join

# Check if directories exit or not
def check_dirs(path_list):
    for dir_path in path_list:
        if not os.path.isdir(dir_path):
            os.makedirs(dir_path)

def check_Q3_fiscal_period(data_path):
    df_cn_en = pd.read_csv(join(data_path, 'cn_to_en.csv'))
    cn_to_en = {c: e for c, e in zip(df_cn_en['cn_name'], df_cn_en['en_name'])}

    save_path = join(data_path, 'Q3-3-not-9_endDate')
    check_dirs([save_path])

    for en_name in cn_to_en.values():
        print('File:', en_name)
        df = pd.read_csv(join(join(data_path, 'original/csv'), en_name) + '.csv')
        df_Q3_3 = df[(df['reportType']=='Q3') | (df['reportType']=='CQ3')].groupby(
            ['ticker', 'endDate']).apply(
                lambda x: 9 not in set(x['fiscalPeriod']))
        df_Q3_3_not_9 = []
        for t, ed in df_Q3_3[df_Q3_3.values].index.to_list():
            df_Q3_3_not_9.append(
                df[(df['ticker']==t) \
                & (df['endDate']==ed) \
                & (df['mergedFlag']==1)])
        if df_Q3_3_not_9:
            df_Q3_3_not_9 = pd.concat(df_Q3_3_not_9)
            df_Q3_3_not_9.to_excel(join(save_path, en_name) + '.xlsx', index=False)

In [11]:
check_Q3_fiscal_period('../../data/financial_statements/')
# check_Q3_fiscal_period('../../data/financial_statements_pit/')

File: income_bank
File: income_insurance
File: income_security
File: income_industry
File: balance_bank
File: balance_insurance
File: balance_security
File: balance_industry
File: cash_flow_bank
File: cash_flow_insurance
File: cash_flow_security
File: cash_flow_industry


# Time series statistics
***

# income - year

In [12]:
# Check if directories exit or not
def check_dirs(path_list):
    for dir_path in path_list:
        if not os.path.isdir(dir_path):
            os.makedirs(dir_path)

def generate_year_revenue_sheets(df_income, data_path, file_name):

    df_income['ticker'] = df_income['ticker'].apply(lambda x: str(x).zfill(6))
    df_income = df_income[df_income['endDate'].apply(lambda x: str(x)[-6:]=='-12-31')]
    df_income.drop_duplicates(subset=['ticker', 'endDate'], keep='last', inplace=True)
    df_income['endDate'] = df_income['endDate'].apply(lambda x: int(x[:4]))

    df_income.set_index(['endDate', 'ticker'], inplace=True)
    df_income.index.names = [None, 'TICKER']

    df_income_r = df_income[['revenue']]
    df_income_r = df_income_r.unstack(1).T
    df_income_r.reset_index(inplace=True)
    df_income_r = df_income_r.drop(columns=['level_0'])
    df_income_r.set_index('TICKER', inplace=True)

    df_income_r_count_list = pd.DataFrame(columns=('TICKER', 'COUNTS', 'YEARS'))
    df_income_r_not_ct_list = pd.DataFrame(columns=('TICKER', 'YEARS'))
    last_year = int(sorted(df_income_r.columns)[-1])

    for ticker, row in df_income_r.iterrows():
        row = row.notnull()
        counts = sum(row)
        years_list = []
        for k in row.keys():
            if row[k]:
                years_list.append(k)
        df_income_r_count_list = df_income_r_count_list.append(
            pd.DataFrame({'TICKER': [ticker], 'COUNTS': [counts], 'YEARS': [years_list]}), ignore_index=True)

        if not years_list:
            print('No data:', ticker)
            df_income_r_not_ct_list = df_income_r_not_ct_list.append(
                pd.DataFrame({'TICKER': [ticker], 'YEARS': [[]]}), ignore_index=True)
        # elif years_list[-1] - years_list[0] != counts - 1:
        elif last_year - years_list[0] != counts - 1:
            df_income_r_not_ct_list = df_income_r_not_ct_list.append(
                pd.DataFrame({'TICKER': [ticker], 'YEARS': [years_list]}), ignore_index=True)

    save_path = join(data_path, 'revenue')
    check_dirs([save_path])
    df_income_r.to_excel(join(save_path, file_name + '.xlsx'), index=True)
    df_income_r_count_list.to_excel(join(save_path, file_name + '_counts.xlsx'), index=False)
    df_income_r_not_ct_list.to_excel(join(save_path, file_name + '_discontinuous.xlsx'), index=False)

In [13]:
data_path = '../../data/financial_statements/normalized_l2/'
df_1 = pd.read_csv(join(join(data_path, 'csv'), 'income_bank.csv'))[['ticker', 'endDate', 'revenue']]
df_2 = pd.read_csv(join(join(data_path, 'csv'), 'income_insurance.csv'))[['ticker', 'endDate', 'revenue']]
df_3 = pd.read_csv(join(join(data_path, 'csv'), 'income_security.csv'))[['ticker', 'endDate', 'revenue']]
df_4 = pd.read_csv(join(join(data_path, 'csv'), 'income_industry.csv'))[['ticker', 'endDate', 'revenue']]
df = pd.concat([df_1, df_2, df_3, df_4])
generate_year_revenue_sheets(df, data_path, 'revenue_year')

# data_path = '../../data/financial_statements_pit/normalized_l2/'
# df_1 = pd.read_csv(join(join(data_path, 'csv'), 'income_bank_pit.csv'))[['ticker', 'endDate', 'revenue']]
# df_2 = pd.read_csv(join(join(data_path, 'csv'), 'income_insurance_pit.csv'))[['ticker', 'endDate', 'revenue']]
# df_3 = pd.read_csv(join(join(data_path, 'csv'), 'income_security_pit.csv'))[['ticker', 'endDate', 'revenue']]
# df_4 = pd.read_csv(join(join(data_path, 'csv'), 'income_industry_pit.csv'))[['ticker', 'endDate', 'revenue']]
# df = pd.concat([df_1, df_2, df_3, df_4])
# generate_year_revenue_sheets(df, data_path, 'revenue_year_pit')

ipykernel_launcher:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
ipykernel_launcher:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


# industry t-revenue - revenue

In [14]:
df_income_industry = pd.read_csv('../../data/financial_statements/normalized_l2/csv/income_industry.csv')
col_name = df_income_industry.columns.tolist()
col_name.insert(col_name.index('revenue') + 1, 'tRevenue-revenue')
df_income_industry = df_income_industry.reindex(columns=col_name)
df_income_industry['tRevenue-revenue'] = df_income_industry['tRevenue'] - df_income_industry['revenue']
df_income_industry.to_excel('../../data/financial_statements/normalized_l2/revenue/income_industry_t_revenue.xlsx')

# df_income_industry = pd.read_csv('../../data/financial_statements_pit/normalized_l2/csv/income_industry_pit.csv')
# col_name = df_income_industry.columns.tolist()
# col_name.insert(col_name.index('revenue') + 1, 'tRevenue-revenue')
# df_income_industry = df_income_industry.reindex(columns=col_name)
# df_income_industry['tRevenue-revenue'] = df_income_industry['tRevenue'] - df_income_industry['revenue']
# df_income_industry.to_excel('../../data/financial_statements_pit/normalized_l2/revenue/income_industry_pit_t_revenue.xlsx')

# income - quarter

In [15]:
# Check if directories exit or not
def check_dirs(path_list):
    for dir_path in path_list:
        if not os.path.isdir(dir_path):
            os.makedirs(dir_path)

def generate_quarter_revenue_sheets(df_income, data_path, file_name):

    df_income['ticker'] = df_income['ticker'].apply(lambda x: str(x).zfill(6))
    df_income.drop_duplicates(subset=['ticker', 'endDate'], keep='last', inplace=True)
    df_income['endDate'] = df_income['endDate'].apply(lambda x: x[:7])

    df_income.set_index(['endDate', 'ticker'], inplace=True)
    df_income.index.names = [None, 'TICKER']

    df_income_r = df_income[['revenue']]
    df_income_r = df_income_r.unstack(1).T
    df_income_r.reset_index(inplace=True)
    df_income_r = df_income_r.drop(columns=['level_0'])
    df_income_r.set_index('TICKER', inplace=True)

    df_income_r_count_list = pd.DataFrame(columns=('TICKER', 'COUNTS', 'QUARTERS'))
    df_income_r_not_ct_list = pd.DataFrame(columns=('TICKER', 'QUARTERS'))
    quarters_all = sorted(df_income_r.columns)
    last_quarter = quarters_all[-1]

    for ticker, row in df_income_r.iterrows():
        row = row.notnull()
        counts = sum(row)
        quarters_list = []
        for k in row.keys():
            if row[k]:
                quarters_list.append(k)
        df_income_r_count_list = df_income_r_count_list.append(
            pd.DataFrame({'TICKER': [ticker], 'COUNTS': [counts], 'QUARTERS': [quarters_list]}), ignore_index=True)

        if not quarters_list:
            print('No data:', ticker)
            df_income_r_not_ct_list = df_income_r_not_ct_list.append(
                pd.DataFrame({'TICKER': [ticker], 'QUARTERS': [[]]}), ignore_index=True)
        # elif quarters_list[-1] - quarters_list[0] != counts - 1:
        elif len(quarters_all) - quarters_all.index(quarters_list[0]) != counts:
            df_income_r_not_ct_list = df_income_r_not_ct_list.append(
                pd.DataFrame({'TICKER': [ticker], 'QUARTERS': [quarters_list]}), ignore_index=True)

    save_path = join(data_path, 'revenue')
    check_dirs([save_path])
    df_income_r.to_excel(join(save_path, file_name + '.xlsx'), index=True)
    df_income_r_count_list.to_excel(join(save_path, file_name + '_counts.xlsx'), index=False)
    df_income_r_not_ct_list.to_excel(join(save_path, file_name + '_discontinuous.xlsx'), index=False)

In [16]:
data_path = '../../data/financial_statements/normalized_l2/'
df_1 = pd.read_csv(join(join(data_path, 'csv'), 'income_bank.csv'))[['ticker', 'endDate', 'revenue']]
df_2 = pd.read_csv(join(join(data_path, 'csv'), 'income_insurance.csv'))[['ticker', 'endDate', 'revenue']]
df_3 = pd.read_csv(join(join(data_path, 'csv'), 'income_security.csv'))[['ticker', 'endDate', 'revenue']]
df_4 = pd.read_csv(join(join(data_path, 'csv'), 'income_industry.csv'))[['ticker', 'endDate', 'revenue']]
df = pd.concat([df_1, df_2, df_3, df_4])
generate_quarter_revenue_sheets(df, data_path, 'revenue_quarter')

# data_path = '../../data/financial_statements_pit/normalized_l2/'
# df_1 = pd.read_csv(join(join(data_path, 'csv'), 'income_bank_pit.csv'))[['ticker', 'endDate', 'revenue']]
# df_2 = pd.read_csv(join(join(data_path, 'csv'), 'income_insurance_pit.csv'))[['ticker', 'endDate', 'revenue']]
# df_3 = pd.read_csv(join(join(data_path, 'csv'), 'income_security_pit.csv'))[['ticker', 'endDate', 'revenue']]
# df_4 = pd.read_csv(join(join(data_path, 'csv'), 'income_industry_pit.csv'))[['ticker', 'endDate', 'revenue']]
# df = pd.concat([df_1, df_2, df_3, df_4])
# generate_quarter_revenue_sheets(df, data_path, 'revenue_quarter_pit')

# Check data absence in three statements
***

In [17]:
import numpy as np
import pandas as pd
from os.path import join
import matplotlib.pyplot as plt

%matplotlib inline
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Check if directories exit or not
def check_dirs(path_list):
    for dir_path in path_list:
        if not os.path.isdir(dir_path):
            os.makedirs(dir_path)

def get_concat_df(data_path, name, post_fix=''):
    df_1 = pd.read_csv(join(join(data_path, 'csv'), name + '_bank' + post_fix + '.csv'))[['ticker', 'endDate']]
    df_2 = pd.read_csv(join(join(data_path, 'csv'), name + '_insurance' + post_fix + '.csv'))[['ticker', 'endDate']]
    df_3 = pd.read_csv(join(join(data_path, 'csv'), name + '_security' + post_fix + '.csv'))[['ticker', 'endDate']]
    df_4 = pd.read_csv(join(join(data_path, 'csv'), name + '_industry' + post_fix + '.csv'))[['ticker', 'endDate']]
    df = pd.concat([df_1, df_2, df_3, df_4])
    df['ticker'] = df['ticker'].apply(lambda x: str(x).zfill(6))
    df = df.set_index(['ticker', 'endDate'])
    return set(df.index.to_list())

In [18]:
data_path = '../../data/financial_statements/normalized_l2/'
set_income = get_concat_df(data_path, 'income')
set_cash_flow = get_concat_df(data_path, 'cash_flow')
set_balance= get_concat_df(data_path, 'balance')

check_dirs([data_path + 'absence/'])

print(len(set_income - set_cash_flow))
with open(data_path + 'absence/income-cash_flow.txt', 'w') as f:
    f.write('Total: {}\n'.format(len(set_income - set_cash_flow)))
    f.write('-' * 50 + '\n')
    for i in sorted(list(set_income - set_cash_flow)):
        f.write(str(i) + '\n')
print(len(set_cash_flow - set_income))
with open(data_path + 'absence/cash_flow-income.txt', 'w') as f:
    f.write('Total: {}\n'.format(len(set_cash_flow - set_income)))
    f.write('-' * 50 + '\n')
    for i in sorted(list(set_cash_flow - set_income)):
        f.write(str(i) + '\n')
print(len(set_income - set_balance))
with open(data_path + 'absence/income-balance.txt', 'w') as f:
    f.write('Total: {}\n'.format(len(set_income - set_balance)))
    f.write('-' * 50 + '\n')
    for i in sorted(list(set_income - set_balance)):
        f.write(str(i) + '\n')
print(len(set_balance - set_income))
with open(data_path + 'absence/balance-income.txt', 'w') as f:
    f.write('Total: {}\n'.format(len(set_balance - set_income)))
    f.write('-' * 50 + '\n')
    for i in sorted(list(set_balance - set_income)):
        f.write(str(i) + '\n')
print(len(set_balance - set_cash_flow))
with open(data_path + 'absence/balance-cash_flow.txt', 'w') as f:
    f.write('Total: {}\n'.format(len(set_balance - set_cash_flow)))
    f.write('-' * 50 + '\n')
    for i in sorted(list(set_balance - set_cash_flow)):
        f.write(str(i) + '\n')
print(len(set_cash_flow - set_balance))
with open(data_path + 'absence/cash_flow-balance.txt', 'w') as f:
    f.write('Total: {}\n'.format(len(set_cash_flow - set_balance)))
    f.write('-' * 50 + '\n')
    for i in sorted(list(set_cash_flow - set_balance)):
        f.write(str(i) + '\n')

# data_path = '../../data/financial_statements_pit/normalized_l2/'
# set_income = get_concat_df(data_path, 'income', '_pit')
# set_cash_flow = get_concat_df(data_path, 'cash_flow', '_pit')
# set_balance= get_concat_df(data_path, 'balance', '_pit')

# check_dirs([data_path + 'absence/'])

# print(len(set_income - set_cash_flow))
# with open(data_path + 'absence/income-cash_flow.txt', 'w') as f:
#     f.write('Total: {}\n'.format(len(set_income - set_cash_flow)))
#     f.write('-' * 50 + '\n')
#     for i in sorted(list(set_income - set_cash_flow)):
#         f.write(str(i) + '\n')
# print(len(set_cash_flow - set_income))
# with open(data_path + 'absence/cash_flow-income.txt', 'w') as f:
#     f.write('Total: {}\n'.format(len(set_cash_flow - set_income)))
#     f.write('-' * 50 + '\n')
#     for i in sorted(list(set_cash_flow - set_income)):
#         f.write(str(i) + '\n')
# print(len(set_income - set_balance))
# with open(data_path + 'absence/income-balance.txt', 'w') as f:
#     f.write('Total: {}\n'.format(len(set_income - set_balance)))
#     f.write('-' * 50 + '\n')
#     for i in sorted(list(set_income - set_balance)):
#         f.write(str(i) + '\n')
# print(len(set_balance - set_income))
# with open(data_path + 'absence/balance-income.txt', 'w') as f:
#     f.write('Total: {}\n'.format(len(set_balance - set_income)))
#     f.write('-' * 50 + '\n')
#     for i in sorted(list(set_balance - set_income)):
#         f.write(str(i) + '\n')
# print(len(set_balance - set_cash_flow))
# with open(data_path + 'absence/balance-cash_flow.txt', 'w') as f:
#     f.write('Total: {}\n'.format(len(set_balance - set_cash_flow)))
#     f.write('-' * 50 + '\n')
#     for i in sorted(list(set_balance - set_cash_flow)):
#         f.write(str(i) + '\n')
# print(len(set_cash_flow - set_balance))
# with open(data_path + 'absence/cash_flow-balance.txt', 'w') as f:
#     f.write('Total: {}\n'.format(len(set_cash_flow - set_balance)))
#     f.write('-' * 50 + '\n')
#     for i in sorted(list(set_cash_flow - set_balance)):
#         f.write(str(i) + '\n')

93
42
8337
10
25
8301


# Check data continuity in three statements
***

In [19]:
import numpy as np
import pandas as pd
from os.path import join
import matplotlib.pyplot as plt

%matplotlib inline
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Check if directories exit or not
def check_dirs(path_list):
    for dir_path in path_list:
        if not os.path.isdir(dir_path):
            os.makedirs(dir_path)

def check_continuity_year(df, data_path, file_name):

    df['ticker'] = df['ticker'].apply(lambda x: str(x).zfill(6))
    df = df[df['endDate'].apply(lambda x: str(x)[-6:]=='-12-31')]
    df.drop_duplicates(subset=['ticker', 'endDate'], keep='last', inplace=True)
    df['endDate'] = df['endDate'].apply(lambda x: int(x[:4]))

    df.set_index(['endDate', 'ticker'], inplace=True)
    df.index.names = [None, 'TICKER']

    df_r = df[['reportType']]
    df_r = df_r.unstack(1).T
    df_r.reset_index(inplace=True)
    df_r = df_r.drop(columns=['level_0'])
    df_r.set_index('TICKER', inplace=True)

    df_r_count_list = pd.DataFrame(columns=('TICKER', 'COUNTS', 'YEARS'))
    df_r_not_ct_list = pd.DataFrame(columns=('TICKER', 'YEARS'))
    last_year = int(sorted(df_r.columns)[-1])

    for ticker, row in df_r.iterrows():
        row = row.notnull()
        counts = sum(row)
        years_list = []
        for k in row.keys():
            if row[k]:
                years_list.append(k)
        df_r_count_list = df_r_count_list.append(
            pd.DataFrame({'TICKER': [ticker], 'COUNTS': [counts], 'YEARS': [years_list]}), ignore_index=True)

        if not years_list:
            print('No data:', ticker)
            df_r_not_ct_list = df_r_not_ct_list.append(
                pd.DataFrame({'TICKER': [ticker], 'YEARS': [[]]}), ignore_index=True)
        # elif years_list[-1] - years_list[0] != counts - 1:
        elif last_year - years_list[0] != counts - 1:
            df_r_not_ct_list = df_r_not_ct_list.append(
                pd.DataFrame({'TICKER': [ticker], 'YEARS': [years_list]}), ignore_index=True)

    save_path = join(data_path, 'continuity')
    check_dirs([save_path])
    df_r_count_list.to_excel(join(save_path, file_name + '_counts.xlsx'), index=False)
    df_r_not_ct_list.to_excel(join(save_path, file_name + '_discontinuous.xlsx'), index=False)

def check_continuity_quarter(df, data_path, file_name):

    df['ticker'] = df['ticker'].apply(lambda x: str(x).zfill(6))
    df.drop_duplicates(subset=['ticker', 'endDate'], keep='last', inplace=True)
    df['endDate'] = df['endDate'].apply(lambda x: x[:7])

    df.set_index(['endDate', 'ticker'], inplace=True)
    df.index.names = [None, 'TICKER']

    df_r = df[['reportType']]
    df_r = df_r.unstack(1).T
    df_r.reset_index(inplace=True)
    df_r = df_r.drop(columns=['level_0'])
    df_r.set_index('TICKER', inplace=True)

    df_r_count_list = pd.DataFrame(columns=('TICKER', 'COUNTS', 'QUARTERS'))
    df_r_not_ct_list = pd.DataFrame(columns=('TICKER', 'QUARTERS'))
    quarters_all = sorted(df_r.columns)
    last_quarter = quarters_all[-1]

    for ticker, row in df_r.iterrows():
        row = row.notnull()
        counts = sum(row)
        quarters_list = []
        for k in row.keys():
            if row[k]:
                quarters_list.append(k)
        df_r_count_list = df_r_count_list.append(
            pd.DataFrame({'TICKER': [ticker], 'COUNTS': [counts], 'QUARTERS': [quarters_list]}), ignore_index=True)

        if not quarters_list:
            print('No data:', ticker)
            df_r_not_ct_list = df_r_not_ct_list.append(
                pd.DataFrame({'TICKER': [ticker], 'QUARTERS': [[]]}), ignore_index=True)
        # elif quarters_list[-1] - quarters_list[0] != counts - 1:
        elif len(quarters_all) - quarters_all.index(quarters_list[0]) != counts:
            df_r_not_ct_list = df_r_not_ct_list.append(
                pd.DataFrame({'TICKER': [ticker], 'QUARTERS': [quarters_list]}), ignore_index=True)

    save_path = join(data_path, 'continuity')
    check_dirs([save_path])
    df_r_count_list.to_excel(join(save_path, file_name + '_counts.xlsx'), index=False)
    df_r_not_ct_list.to_excel(join(save_path, file_name + '_discontinuous.xlsx'), index=False)

In [20]:
def get_concat_df(data_path, name, post_fix=''):
    df_1 = pd.read_csv(join(join(data_path, 'csv'), name + '_bank' + post_fix + '.csv'))[['ticker', 'endDate', 'reportType']]
    df_2 = pd.read_csv(join(join(data_path, 'csv'), name + '_insurance' + post_fix + '.csv'))[['ticker', 'endDate', 'reportType']]
    df_3 = pd.read_csv(join(join(data_path, 'csv'), name + '_security' + post_fix + '.csv'))[['ticker', 'endDate', 'reportType']]
    df_4 = pd.read_csv(join(join(data_path, 'csv'), name + '_industry' + post_fix + '.csv'))[['ticker', 'endDate', 'reportType']]
    df = pd.concat([df_1, df_2, df_3, df_4])
    return df

data_path = '../../data/financial_statements/normalized_l2/'
df_income = get_concat_df(data_path, 'income')
df_cash_flow = get_concat_df(data_path, 'cash_flow')
df_balance= get_concat_df(data_path, 'balance')

check_continuity_year(df_income, data_path, 'income_year')
check_continuity_year(df_cash_flow, data_path, 'cash_flow_year')
check_continuity_year(df_balance, data_path, 'balance_year')

check_continuity_quarter(df_income, data_path, 'income_quarter')
check_continuity_quarter(df_cash_flow, data_path, 'cash_flow_quarter')
check_continuity_quarter(df_balance, data_path, 'balance_quarter')

ipykernel_launcher:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
ipykernel_launcher:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


# Check duplicates
***

In [21]:
import pandas as pd
from os.path import join
%matplotlib inline
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Check if directories exit or not
def check_dirs(path_list):
    for dir_path in path_list:
        if not os.path.isdir(dir_path):
            os.makedirs(dir_path)

def get_concat_df(data_path, name):
    def _load_csv(category):
        df = pd.read_csv(join(data_path, name + '_' + category + '.csv'), dtype={'ticker': str})
        df['from'] = [category] * df.shape[0]
        return df
    df_1 = _load_csv('bank')
    df_2 = _load_csv('insurance')
    df_3 = _load_csv('security')
    df_4 = _load_csv('industry')
    df = pd.concat([df_1, df_2, df_3, df_4])
    df = df.reset_index()
    df.drop(columns=['index'], inplace=True)
    return df

def check_duplicates(data_path, statement_name):
    df = get_concat_df(join(data_path, 'csv'), statement_name)

    selected = df.groupby(['ticker', 'endDate']).apply(lambda x: tuple(x.index))

    idx_dup = []
    for i in selected.values:
        if len(i) > 1:
            idx_dup.extend(i)
    df_dup = df.loc[idx_dup]

    col_name = df_dup.columns.tolist()
    list1 = ['ticker', 'secShortName', 'from', 'endDate', 'endDateRep', 'actPubtime',  'reportType'] 
    df_dup = df_dup.reindex(columns=list1 + [i for i in col_name if i not in list1])

    df_dup = df_dup.sort_values(
        by=['ticker', 'endDate', 'from', 'endDateRep', 'actPubtime'], 
        ascending=[True, True, True, True, True]
    )

    save_path = join(data_path, 'duplicates')
    check_dirs([save_path])
    df_dup.to_excel(join(save_path, statement_name) + '.xlsx', index=False)
    print(statement_name + ':', df_dup.shape)

In [22]:
data_path = '../../data/financial_statements/normalized_l2'
check_duplicates(data_path, 'income')
check_duplicates(data_path, 'balance')
check_duplicates(data_path, 'cash_flow')

income: (128, 93)
balance: (26, 177)
cash_flow: (190, 89)


# Get all stocks list    

In [1]:
import numpy as np
import pandas as pd
from os.path import join
import matplotlib.pyplot as plt

%matplotlib inline
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [2]:
def get_concat_df(data_path, name):
    def _load_csv(category):
        df = pd.read_csv(join(data_path, name + '_' + category + '.csv'), dtype={'ticker': str})
        return df
    df_1 = _load_csv('bank')[['ticker', 'secShortName']]
    df_2 = _load_csv('insurance')[['ticker', 'secShortName']]
    df_3 = _load_csv('security')[['ticker', 'secShortName']]
    df_4 = _load_csv('industry')[['ticker', 'secShortName']]
    df = pd.concat([df_1, df_2, df_3, df_4])
    return df

### Companies in statements

In [3]:
data_path = '../../data/financial_statements/normalized_l2'
df_income = get_concat_df(join(data_path, 'csv'), 'income')
df_balance = get_concat_df(join(data_path, 'csv'), 'balance')
df_cash_flow = get_concat_df(join(data_path, 'csv'), 'cash_flow')
df = pd.concat([df_income, df_balance, df_cash_flow])
set_s = set(df['ticker'])
dict_s = {t: n for t, n in zip(df['ticker'], df['secShortName'])}

In [4]:
len(set_s)

4307

### Listed time >= 20180101 and Unlisted

In [2]:
df_info = pd.read_csv('../../data/stocks_information/stocks_info_20210417.csv', dtype={'ticker': str})


In [3]:
new_listed = set(df_info[(df_info['listDate'] >= '2018-01-01') | (df_info['listStatusCD'] == 'UN') | (df_info['listStatusCD'] == 'O')]['ticker'])

In [11]:
new_listed = set(df_info[(df_info['listStatusCD'] == 'UN') | (df_info['listStatusCD'] == 'O')]['ticker'])
[i for i in new_listed if i[0] not in ['A', '9', '2']]

['688655',
 '605080',
 '300982',
 '001201',
 '688323',
 '000991',
 '001203',
 '600906',
 '002710',
 '002525',
 '605016',
 '002720',
 '601206',
 '605289',
 '300728',
 '605098',
 '300060',
 '605117',
 '688097',
 '300975',
 '300978',
 '605196',
 '688682',
 '688201',
 '688383',
 '300980',
 '300972',
 '688533',
 '300646',
 '001202',
 '688395',
 '688113',
 '605305',
 '603302',
 '600349',
 '688639',
 '300983',
 '300986',
 '605300',
 '605089',
 '300361',
 '300976',
 '300981',
 '600032',
 '688688',
 '300977',
 '300985',
 '300987',
 '605180',
 '603511',
 '002257',
 '300979']

### Companies in classification_fixed

In [7]:
df_cls = pd.read_csv('../../data/classification/classification_fixed.csv', dtype={'TICKER_SYMBOL': str})
set_c = set(df_cls['TICKER_SYMBOL'])
dict_c = {t: n for t, n in zip(df_cls['TICKER_SYMBOL'], df_cls['STOCK_NAME'])}

In [8]:
len(set_c)

3521

### Companies in cal_now_values

In [14]:
df = pd.read_csv('../../data/classification/old/cal_now_values.csv', dtype={'TICKER_SYMBOL': str})
df['ticker'] = df['TICKER_SYMBOL'].apply(lambda x: str(x).zfill(6))
set_o = set(df['ticker'])

In [15]:
len(set_o)

3521

### Get intersection

In [16]:
for i in sorted([(i, dict_s[i]) for i in set_s - new_listed - set_c]):
    print(i)

('001872', '招商港口')
('001914', '招商积余')
('002808', '恒久科技')


In [17]:
with open(data_path + '/absence/all_tickers-classification.txt', 'w') as f:
    f.write('Total: {}\n'.format(len(set_s - new_listed - set_c)))
    f.write('-' * 50 + '\n')
    for i in sorted([(i, dict_s[i]) for i in set_s - new_listed - set_c]):
        f.write(str(i) + '\n')

In [18]:
len(set_s - new_listed)

3429

In [19]:
set_o - set_c

set()

In [3]:
df_balance = pd.read_csv('../../data/standard_dataset/total_sheet_3(standard_2)/csv/balance_general.csv')[['ticker', 'endDate']]
df_cash_flow = pd.read_csv('../../data/standard_dataset/total_sheet_3(standard_2)/csv/cash_flow_general.csv')[['ticker', 'endDate']]
df_income = pd.read_csv('../../data/standard_dataset/total_sheet_3(standard_2)/csv/income_general.csv')[['ticker', 'endDate']]

In [7]:
df_all = pd.concat([df_balance, df_cash_flow, df_income]).drop_duplicates()
df_all['ticker'] = df_all['ticker'].apply(lambda x: str(x).zfill(6))

In [29]:
df_all[df_all['ticker'] == '600093']

,ticker,endDate
96445,600093,2007-03-31
96446,600093,2007-06-30
96447,600093,2007-09-30
96448,600093,2007-12-31
96449,600093,2008-03-31
96450,600093,2008-06-30
96451,600093,2008-09-30
96452,600093,2008-12-31
96453,600093,2009-03-31
96454,600093,2009-06-30


In [25]:
comp = []
for t, f in df_all.groupby('ticker').apply(lambda x: ('2020-12-31' not in tuple(x['endDate'])) and (len(tuple(x['endDate'])) > 0)).iteritems():
    if f and t not in df_delist['TICKER'].values:
        comp.append(t)


In [26]:
comp

['000760', '002359', '600093']

In [24]:
'000003' in df_delist['TICKER'].values

True